In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [2]:
from datetime import datetime, timedelta
from pandas import DataFrame, to_datetime, set_option
from TpsParse.Tps.TpsFile import TpsFile
import os
from numpy import quantile, nan
from math import ceil
import pandas as pd

set_option('display.max_columns', 500)
set_option('display.max_row', 1000)

from bokeh.resources import CDN
from bokeh.embed import components
from bokeh.models import Range1d, NumeralTickFormatter, ColumnDataSource, FactorRange, HoverTool, \
DataRange1d, DatetimeTickFormatter
from bokeh.plotting import figure
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.io import show, output_notebook
from bokeh.palettes import Category20_20

from django.db.models import Sum, DateField
from django.db.models.functions import Trunc
from Utils.clean_string import clean_string

output_notebook()

Loading BokehJS ...

In [3]:
def parser(path):
    file = open(path, "r+b")
    tps = TpsFile(file)

    cols_and_type = []
    columns = []
    records = []
    for definition in tps.get_table_definitions():
        for field in definition.fields:
            column = field.field_name.split(':')[1].lower()
            col_and_type = (field.field_name.split(':')[1].lower(), field.type)
            if column not in columns:
                columns.append(column)
                cols_and_type.append(col_and_type)
        for record in tps.get_data_records(definition):
            records.append([r.strip().title() if isinstance(r, str) else r for r in record.values])

    table = []
    for record in records:
        dic = {}
        for col, rec in zip(columns, record):
            dic[col] = rec
        table.append(dic)

    df = DataFrame(table, columns=columns)

    name_list = ('fec', 'periodo')
    # Formatear la fecha.
    date_columns = [cname for cname, ctype in cols_and_type if cname.startswith(name_list) and ctype == 'SignedLong']
    for col in df[date_columns]:
        df[col] = datetime(1800, 12, 28) + df[col].map(timedelta)
        df.loc[df[col] == '1800-12-28', col] = None

    # Reemplazar los campos vacíos con None.
    df.replace(to_replace='', value=nan, inplace=True)
    df.replace(to_replace=0, value=None, inplace=True)
    
    # Borrar los registros que tengan todos valores nulos.
    df.dropna(axis='index', how='all', inplace=True)
    
    # Eliminar registros duplicados.
    df.drop_duplicates(keep='last', inplace=True)
    
    # Borrar las columnas que tengan todos valores nulos o ceros.
    df.dropna(axis='columns', how='all', inplace=True)
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df

In [4]:
path = "/home/wen/Documentos/Proyecto/Datos Agrosoft/CTA_CTE.TPS"
df = parser(path);df

,tipo_movim,cod_proveedor,fecha_comprob,nro_comprob,importe,periodo_contable,prefijo,tipo_comprobante
0,1,34,2011-11-24,241111,520.00,2011-11-24,0,NaN
1,1,34,2011-11-24,241111,36.15,2011-11-24,0,NaN
2,1,34,2011-10-13,131011,47.24,2011-10-13,0,NaN
3,1,35,2011-12-30,934,5327.26,2011-12-30,0,NaN
4,1,35,2011-12-30,236,11381.00,2011-12-30,0,NaN
5,1,36,2011-11-21,21705,687.25,2011-12-30,0,NaN
6,1,17,2011-12-04,153285,100.00,2011-12-30,0,NaN
7,1,17,2011-11-18,152576,100.00,2011-12-30,0,NaN
8,1,17,2011-12-31,154750,218.00,2011-12-31,0,NaN
9,1,17,2011-12-06,153347,100.00,2011-12-31,0,NaN


In [6]:
from app.models import CtaCte

cta_cte = CtaCte.objects.filter(cod_proveedor__razon_social__contains='Bonafede Emiliana')\
    .values(
    'tipo_movim__descripcion',
    'cod_proveedor',
    'cod_proveedor__razon_social',
    'cod_proveedor__direccion',
    'cod_proveedor__localidad__descripcion',
    'cod_proveedor__cuit',
    'cod_proveedor__saldo_cta_cte',
    'fecha_comprob',
    'nro_comprob',
    'importe',
    'periodo_contable',
    'tipo_comprobante')

df = DataFrame(cta_cte)
df = df.rename({'cod_proveedor__cuit': 'cuit', 
               'cod_proveedor__direccion': 'direccion',
               'cod_proveedor__localidad__descripcion': 'localidad', 
               'cod_proveedor__razon_social': 'razon_social', 
               'tipo_movim__descripcion': 'tipo_movimiento', 
                'cod_proveedor__saldo_cta_cte': 'saldo'}, axis='columns')
df.sort_values('fecha_comprob', inplace=True)
df['fecha_comprob'] = to_datetime(df.fecha_comprob)

In [8]:
df['debe'] = df.loc[df.tipo_movimiento == 'Orden de Pago'].importe
df['haber'] = df.loc[df.tipo_movimiento == 'Factura Cuenta Corriente'].importe; df

,tipo_movimiento,cod_proveedor,razon_social,direccion,localidad,cuit,saldo,fecha_comprob,nro_comprob,importe,periodo_contable,tipo_comprobante,debe,haber
50,Factura Cuenta Corriente,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-01-28,0,5000.0,2015-01-28,None,NaN,5000.0
52,Orden de Pago,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-01-30,2533,8000.0,None,None,8000.0,NaN
51,Factura Cuenta Corriente,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-01-30,1,3000.0,2015-01-30,None,NaN,3000.0
4,Factura Cuenta Corriente,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-02-01,2,3000.0,2015-02-01,None,NaN,3000.0
7,Orden de Pago,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-02-10,2703,3000.0,None,None,3000.0,NaN
5,Factura Cuenta Corriente,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-03-01,3,3000.0,2015-03-01,None,NaN,3000.0
8,Orden de Pago,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-03-10,2704,3000.0,None,None,3000.0,NaN
6,Factura Cuenta Corriente,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-04-01,4,3000.0,2015-04-01,None,NaN,3000.0
9,Orden de Pago,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-04-10,2705,3000.0,None,None,3000.0,NaN
13,Factura Cuenta Corriente,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-05-01,5,3000.0,2015-05-01,None,NaN,3000.0


In [17]:
df = df.where((pd.notnull(df)), None); df

,tipo_movimiento,cod_proveedor,razon_social,direccion,localidad,cuit,saldo,fecha_comprob,nro_comprob,importe,periodo_contable,tipo_comprobante,debe,haber
452,Factura Cuenta Corriente,72,Herminio Bonafede (Retiro),None,Colonia Aldao,11111111111,-12387.8,2012-03-27,24335636,768.78,2012-04-30,None,None,768.78
293,Orden de Pago,73,Eloy Bonafede (Retiro),None,Colonia Aldao,11111111111,-16049,2012-04-03,58,2100,None,None,2100,None
292,Factura Cuenta Corriente,73,Eloy Bonafede (Retiro),None,Colonia Aldao,11111111111,-16049,2012-04-03,3042012,2100,2012-04-30,None,None,2100
290,Factura Cuenta Corriente,93,Hugo Bonafede (Retiro),None,Colonia Aldao,11111111111,-3045.02,2012-04-09,9042012,1210,2012-04-30,None,None,1210
291,Orden de Pago,93,Hugo Bonafede (Retiro),None,Colonia Aldao,11111111111,-3045.02,2012-04-09,54,1210,None,None,1210,None
449,Contado,72,Herminio Bonafede (Retiro),None,Colonia Aldao,11111111111,-12387.8,2012-04-17,17042012,800,None,None,None,None
457,Orden de Pago,73,Eloy Bonafede (Retiro),None,Colonia Aldao,11111111111,-16049,2012-04-17,78,5188.27,None,None,5188.27,None
456,Factura Cuenta Corriente,73,Eloy Bonafede (Retiro),None,Colonia Aldao,11111111111,-16049,2012-04-17,17042012,5188.27,2012-04-30,None,None,5188.27
455,Orden de Pago,93,Hugo Bonafede (Retiro),None,Colonia Aldao,11111111111,-3045.02,2012-04-17,70,621.62,None,None,621.62,None
454,Factura Cuenta Corriente,93,Hugo Bonafede (Retiro),None,Colonia Aldao,11111111111,-3045.02,2012-04-17,17042012,621.62,2012-04-30,None,None,621.62


In [5]:
debe = df[(df.fecha_comprob >= '1-18-2017') & (df.fecha_comprob <= '2-1-2017')].copy(); debe

,tipo_movimiento,cod_proveedor,razon_social,direccion,localidad,cuit,saldo,fecha_comprob,nro_comprob,importe,periodo_contable,tipo_comprobante
381,Factura Cuenta Corriente,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2017-01-18,1,6000.0,2017-01-18,None
364,Orden de Pago,367,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2017-01-19,4179,6000.0,None,None
408,Factura Cuenta Corriente,221,Analia G. Bonafede,San Martin 422,Sunchales,27172742993,142400.0,2017-01-21,64,7818.3,2017-04-01,None
359,Orden de Pago,221,Analia G. Bonafede,San Martin 422,Sunchales,27172742993,142400.0,2017-01-21,4140,7818.3,None,None
395,Factura Cuenta Corriente,364,Bonafede Luciano (Sueldo),None,Colonia Aldao,11111111111,-34500.0,2017-01-30,1,3750.0,2017-01-30,None
360,Orden de Pago,364,Bonafede Luciano (Sueldo),None,Colonia Aldao,11111111111,-34500.0,2017-01-31,4142,3750.0,None,None


In [44]:
debe = df[(df.tipo_movimiento == 'Orden de Pago') & (df.fecha_comprob > '1-1-2017')].copy(); debe

,cod_proveedor,cuit,direccion,localidad,razon_social,saldo,fecha_comprob,importe,nro_comprob,periodo_contable,tipo_comprobante,tipo_movimiento
35,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2017-01-19,6000.0,4179,None,None,Orden de Pago
36,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2017-02-23,4500.0,4212,None,None,Orden de Pago
37,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2017-03-14,4500.0,4251,None,None,Orden de Pago
53,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2017-04-20,4500.0,4317,None,None,Orden de Pago
54,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2017-05-22,4500.0,4378,None,None,Orden de Pago
55,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2017-06-15,4500.0,4444,None,None,Orden de Pago


In [24]:
haber = df[df.tipo_movimiento == 'Factura Cuenta Corriente'].copy(); haber

,cod_proveedor,cuit,direccion,localidad,razon_social,saldo,fecha_comprob,importe,nro_comprob,periodo_contable,tipo_comprobante,tipo_movimiento
50,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2015-01-28,5000.0,0,2015-01-28,None,Factura Cuenta Corriente
51,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2015-01-30,3000.0,1,2015-01-30,None,Factura Cuenta Corriente
4,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2015-02-01,3000.0,2,2015-02-01,None,Factura Cuenta Corriente
5,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2015-03-01,3000.0,3,2015-03-01,None,Factura Cuenta Corriente
6,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2015-04-01,3000.0,4,2015-04-01,None,Factura Cuenta Corriente
13,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2015-05-01,3000.0,5,2015-05-01,None,Factura Cuenta Corriente
12,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2015-06-01,3000.0,6,2015-06-01,None,Factura Cuenta Corriente
15,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2015-07-01,3000.0,7,2015-07-01,None,Factura Cuenta Corriente
16,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2015-08-01,3000.0,8,2015-08-01,None,Factura Cuenta Corriente
18,367,27111111111,25 De Mayo 201,Colonia Aldao,Bonafede Emiliana (Sueldo),-38000.0,2015-09-01,3000.0,9,2015-09-01,None,Factura Cuenta Corriente
